In [1]:
%load_ext autoreload
%autoreload 2



from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score
# noinspection PyUnresolvedReferences
from hcve_lib.tracking import load_run_results
from hcve_lib.utils import notebook_init
from hcve_lib.evaluation_functions import map_inverse_weight


notebook_init()


from notebooks.deps.binary_predictive_performance import run_roc_analysis, get_pr_analysis
from notebooks.deps.binary_predictive_performance import get_pr_analysis, get_pr_analysis_ci, plot_pr_ci

from mlflow import set_tracking_uri
from notebooks.deps.config import TIME_POINT_PREDICTION
from deps.common import get_data_cached

from sklearn.metrics import roc_curve
from pandas import DataFrame
from plotly.graph_objs import Figure

from deps.constants import RANDOM_STATE
from hcve_lib.evaluation_functions import average_group_scores, merge_standardize_prediction, merge_predictions, \
    compute_metrics_prediction
from hcve_lib.metrics import BootstrappedMetric
from hcve_lib.tracking import load_group_results
from hcve_lib.visualisation import setup_plotly_style
from hcve_lib.functional import t
import numpy as np
from hcve_lib.metrics import statistic_from_bootstrap
from hcve_lib.functional import reject_none
from plotly import express as px
from hcve_lib.utils import transpose_list
from notebooks.deps.binary_predictive_performance import run_pr_analysis_ci

from config import GROUPS, GROUPS_10_fold
from hcve_lib.metrics import BinaryMetricFromScore
from hcve_lib.data import binarize_event

set_tracking_uri('http://localhost:5000')

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data, metadata, X, y = get_data_cached()


[Memory]2.9s, 0.0min    : Loading get_data...
____________________________________________get_data cache loaded - 0.0s, 0.0min


In [3]:
TIME_POINT_PREDICTION = 10*365
ITERATIONS = 1

## Counting cases/controls with binarization at end-point

### Total

In [4]:
binarized = binarize_event(TIME_POINT_PREDICTION, y['data'], drop_censored=False)

print('Before')
print(y['data']['label'].value_counts().to_dict())
print('↓')
print(f'Binarization {TIME_POINT_PREDICTION/365} years (NA removed)')
print((binarized.value_counts() - y['data']['label'].value_counts()).to_dict())
print('↓')
print(binarized.value_counts(dropna=True).to_dict())


Before
{0.0: 29286, 1.0: 1068}
↓
Binarization 10.0 years (NA removed)
{0.0: -27294, 1.0: -177}
↓
{0.0: 1992, 1.0: 891}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


In [5]:
from toolz import valmap 

merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)


In [6]:
y_binarized = binarize_event(10*365, y['data'])

/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


### Per cohort

In [7]:
from hcve_lib.visualisation import b
from hcve_lib.data import binarize_event
from hcve_lib.utils import is_noneish

In [8]:
from deps.data import group_by_study

for study, study_X in group_by_study(data, X):
    b(study)
    binarized = binarize_event(TIME_POINT_PREDICTION, y['data'].loc[study_X.index], drop_censored=False)
    print('Before')
    before = y['data'].loc[study_X.index]['label'].value_counts()
    print(before.to_dict())
    print('↓')
    print(f'Binarization {TIME_POINT_PREDICTION/365} years (NA removed)')
    after = (binarized.value_counts() - before).to_dict()
    for value, count in after.items():
        if is_noneish(count):
            after[value] = - before[value]
    print(after)
    print('↓')
    print(binarized.value_counts(dropna=True).to_dict())

Before
{0.0: 19010, 1.0: 243}
↓
Binarization 10.0 years (NA removed)
{0.0: -19010, 1.0: 0.0}
↓
{1.0: 243}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


Before
{0.0: 1083, 1.0: 18}
↓
Binarization 10.0 years (NA removed)
{0.0: -882, 1.0: 0}
↓
{0.0: 201, 1.0: 18}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


Before
{0.0: 2361, 1.0: 574}
↓
Binarization 10.0 years (NA removed)
{0.0: -570, 1.0: -177}
↓
{0.0: 1791, 1.0: 397}


...

/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


Before
{0.0: 1432, 1.0: 31}
↓
Binarization 10.0 years (NA removed)
{0.0: -1432, 1.0: 0.0}
↓
{1.0: 31}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


Before
{0.0: 4975, 1.0: 191}
↓
Binarization 10.0 years (NA removed)
{0.0: -4975, 1.0: 0.0}
↓
{1.0: 191}


/home/sitnarf/projects/hcve_lib/hcve_lib/data.py:249: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_binary = Series(index=survival_y.index.copy())


## PR analysis

In [9]:
pr_metrics_unweighted = (
    BinaryMetricFromScore(precision_recall_curve, time=TIME_POINT_PREDICTION, sample_weight=None),
    BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=None),
)


In [10]:
run_pr_analysis_ci(GROUPS, y, metrics=pr_metrics_unweighted, standardize=False, iterations=ITERATIONS).show()

5e37865c-7821-4726-a015-56447f4b172d
[Memory]22.4s, 0.4min   : Loading load_group_results...
_________________________________load_group_results cache loaded - 41.3s, 0.7min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])


KeyboardInterrupt: 

In [12]:
inverse_weight_cohorts = map_inverse_weight(data['STUDY'])

pr_metrics_weighted_cohorts = [
    BinaryMetricFromScore(precision_recall_curve, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight_cohorts),
    BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight_cohorts),
]

In [ ]:
inverse_incident_weight = map_inverse_weight(binarized, proportions={1:0.03, 0: 0.97})
combined_weight = (inverse_weight_cohorts*inverse_incident_weight).dropna()

pr_metrics_incidence_cohort_weighted = [
    BinaryMetricFromScore(precision_recall_curve, time=TIME_POINT_PREDICTION, sample_weight=combined_weight),
    BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=combined_weight),
]

In [ ]:

pr_metrics_incidence_weighted = [
    BinaryMetricFromScore(precision_recall_curve, time=TIME_POINT_PREDICTION, sample_weight=inverse_incident_weight),
    BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=inverse_incident_weight),
]

In [ ]:
import pandas
pandas.set_option('display.max_rows', 500)

In [ ]:
run_pr_analysis_ci(GROUPS, y, metrics=pr_metrics_weighted_cohorts, standardize=False, iterations=ITERATIONS).show()

In [ ]:
del GROUPS['coxnet_ns']
del GROUPS['gb_s']

In [ ]:
run_pr_analysis_ci(GROUPS, y, metrics=pr_metrics_weighted_cohorts, standardize=False, iterations=ITERATIONS).show()

In [29]:
pr['stacking']['average_precision_score_3650']

[0.6138647569533737,
 0.6668516270611669,
 0.6226636844095713,
 0.6827852398634564,
 0.678642700187379]

In [30]:
pr = get_pr_analysis(
    GROUPS,
    y=y,
    metrics=[
        BinaryMetricFromScore(precision_recall_curve, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight_cohorts),
        BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight_cohorts),
    ],
    standardize=False,
    iterations=5,
)

5e37865c-7821-4726-a015-56447f4b172d
[Memory]2462.3s, 41.0min: Loading load_group_results...



KeyboardInterrupt



In [33]:
pr_summary = get_pr_analysis(
    GROUPS,
    y=y,
    metrics=[
        BinaryMetricFromScore(average_precision_score, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight_cohorts),
    ],
    standardize=False,
    iterations=5,
    return_summary=True,
)

5e37865c-7821-4726-a015-56447f4b172d
[Memory]3264.6s, 54.4min: Loading load_group_results...
_________________________________load_group_results cache loaded - 80.5s, 1.3min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])
values=[0.6138647569533737, 0.6668516270611669, 0.6226636844095713, 0.6827852398634564, 0.678642700187379]
bbcbe7fa-7499-4464-a21e-bb7c6c571886
[Memory]3347.0s, 55.8min: Loading load_group_results...
_________________________________load_group_results cache loaded - 22.6s, 0.4min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])
values=[0.44727893462410107, 0.49304782837762207, 0.4509042845925425, 0.43326166130560173, 0.4750166978363898]
de06404b-c3c4-45e1-a641-e7bde8860f85
[Memory]3370.8s, 56.2min: Loading load_group_results...
_________________________________load_group_results cache loaded - 55.3s, 0.9min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTO

In [34]:
pr_summary

{'stacking': {'average_precision_score_3650': {'mean': 0.6529616016949895,
   'ci': (0.6147446496989935, 0.6823709858958487),
   'std': 0.03736499287941655}},
 'coxnet': {'average_precision_score_3650': {'mean': 0.4599018813472514,
   'ci': (0.4346633886374517, 0.49124471532349884),
   'std': 0.027550610018919848}},
 'coxnet_ns': {'average_precision_score_3650': {'mean': 0.2797043821574114,
   'ci': (0.2567882125634938, 0.29835672932941165),
   'std': 0.018965848360838627}},
 'gb': {'average_precision_score_3650': {'mean': 0.49680626765902647,
   'ci': (0.46825091499899746, 0.5271068367246051),
   'std': 0.02851501940701356}},
 'gb_s': {'average_precision_score_3650': {'mean': 0.4943032668971874,
   'ci': (0.4793115860555934, 0.5241834720493919),
   'std': 0.02279885328974051}},
 'pcp_hf': {'average_precision_score_3650': {'mean': 0.45895069840454505,
   'ci': (0.4281377792572722, 0.4865922802396244),
   'std': 0.027400863095897943}}}

In [15]:
pr = get_pr_analysis(GROUPS, y=y, metrics=pr_metrics_weighted_cohorts, standardize=False, iterations=5)
pr_ci = get_pr_analysis_ci(pr)

5e37865c-7821-4726-a015-56447f4b172d
[Memory]931.5s, 15.5min : Loading load_group_results...
_________________________________load_group_results cache loaded - 40.7s, 0.7min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])
bbcbe7fa-7499-4464-a21e-bb7c6c571886
[Memory]974.1s, 16.2min : Loading load_group_results...
_________________________________load_group_results cache loaded - 10.9s, 0.2min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])
de06404b-c3c4-45e1-a641-e7bde8860f85
[Memory]986.4s, 16.4min : Loading load_group_results...
_________________________________load_group_results cache loaded - 29.3s, 0.5min
result.keys()=dict_keys(['ASCOT', 'FLEMENGHO', 'HEALTHABC', 'HVC', 'PREDICTOR', 'PROSPER'])
52e729f7-0ae3-40a4-b639-8f9f3eaa3a36
[Memory]1017.5s, 17.0min: Loading load_group_results...
_________________________________load_group_results cache loaded - 40.4s, 0.7min
result.keys()=dict_keys(['AS

/home/sitnarf/projects/hcve_lib/hcve_lib/metrics.py:191: RuntimeWarning: divide by zero encountered in double_scalars
  std=(np.sum(
/home/sitnarf/projects/hcve_lib/hcve_lib/metrics.py:191: RuntimeWarning: invalid value encountered in double_scalars
  std=(np.sum(


In [16]:
from hcve_lib.functional import dict_subset

pr_ci_selected = dict_subset(('coxnet', 'gb', 'pcp_hf', 'stacking'), pr_ci)


In [25]:
pr_ci_selected

{'coxnet': ([{'mean': 0.24860863438326616,
    'ci': (0.24485590852164446, 0.2521803049662107),
    'std': 0.003353670090458442},
   {'mean': 0.24660637689364054,
    'ci': (0.24291162995974538, 0.2506834153029819),
    'std': 0.004219609838851066},
   {'mean': 0.24691035602825026,
    'ci': (0.24330894061934483, 0.25092695553210864),
    'std': 0.004226150796177233},
   {'mean': 0.2472558928250928,
    'ci': (0.24354025832466994, 0.25114812762141536),
    'std': 0.004166110191015579},
   {'mean': 0.24756148101450917,
    'ci': (0.24393962757820614, 0.25139261405003166),
    'std': 0.004172877688705452},
   {'mean': 0.24786414763438636,
    'ci': (0.24432512895715444, 0.25161460970014843),
    'std': 0.0040613571884433606},
   {'mean': 0.24830229843127077,
    'ci': (0.24495076757137021, 0.25205977947011066),
    'std': 0.004020774323487789},
   {'mean': 0.24864151690539443,
    'ci': (0.2453838461863925, 0.2524563772870574),
    'std': 0.0038993054529241553},
   {'mean': 0.24907954332

In [41]:
from config import COLORS, get_color
fig = plot_pr_ci(pr_ci_selected)
fig.write_image('./data/precision_recall.svg')
fig.show()

In [ ]:
from notebooks.deps.binary_predictive_performance import get_endpoint_analysis_result

result = get_endpoint_analysis_result(GROUPS, standardize=False)

In [ ]:
y_binarized = binarize_event(TIME_POINT_PREDICTION, y['data'])

In [ ]:
from hcve_lib.utils import binarize
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from hcve_lib.utils import run_parallel

def plot_():
    cm = [
        ['', 'PP', 'PN'],
        ['TP', *cm[0]],
        ['TN', *cm[1]],
    ]
    print(tabulate(cm, tablefmt='fancy_grid'))
    print()

prediction = result['stacking']
y_true = y_binarized

def get_cm_by_threshold(threshold):
    print('.')
    y_pred = binarize(prediction['y_score'], threshold)
    cm = confusion_matrix(y_true, y_pred.loc[y_true.index])
    print(threshold)
    return cm

cm_by_threshold = run_parallel(
    get_cm_by_threshold,
    {threshold: [threshold] for threshold in sorted(set(prediction['y_score']))}
)
    

In [ ]:
!for pid in `pgrep -f jupyter`; do { renice -20 $pid; }; done

In [13]:
pr = get_pr_analysis(GROUPS, y=y, metrics=pr_metrics_weighted_cohorts, standardize=False, iterations=1)

5e37865c-7821-4726-a015-56447f4b172d
[Memory]85.5s, 1.4min   : Loading load_group_results...



KeyboardInterrupt



In [14]:
pr_ci = get_pr_analysis_ci(pr)

NameError: name 'pr' is not defined

In [ ]:
from hcve_lib.tracking import load_pickled_artifact
from mlflow.exceptions import RestException
from hcve_lib.custom_types import ExceptionValue
from mlflow import search_runs

def load_cm_for_threshold(group_id):
    try:
        runs = search_runs(
            filter_string=f'tags.mlflow.runName = "{group_id}"',
            search_all_experiments=True,
            output_format='list',
        )
        return load_pickled_artifact(runs[0].info.run_id, 'result')
    except RestException as e:
        return ExceptionValue(exception=e)

cm_per_threshold = run_pararell(
    load_cm_for_threshold,
    GROUPS,
)

In [ ]:
pr_ci_out = pr_ci
COLORS = {'gb': 'red', 'coxnet': 'blue', 'pcp_hf': 'orange', 'stacking': 'black'}
method_names = []
precision_mean = []
precision_ci_low = []
precision_ci_high = []
recall_mean = []
threshold_mean = []

for method_name, (precision_curve_ci, recall_curve_ci, thresholds) in pr_ci_out.items():
    n_points = len(precision_curve_ci)
    method_names += [method_name] * n_points
    recall_mean += [point['mean'] for point in recall_curve_ci]
    precision_ci_low += [point['ci'][0] for point in precision_curve_ci]
    precision_ci_high += [point['ci'][1] for point in precision_curve_ci]
    precision_mean += [point['mean'] for point in precision_curve_ci]
    threshold_mean += [0]+[point['mean'] for point in thresholds]


pr_ci_df = DataFrame(
    {
        'method_name': method_names,
        'precision_mean': precision_mean,
        'precision_ci_low': precision_ci_low,
        'precision_ci_high': precision_ci_high,
        'recall_mean': recall_mean,
        'threshold': threshold_mean,
        'confusion_matrix': None,
    }
)

pr_ci_df = pr_ci_df.sort_values('recall_mean')



In [ ]:
pr_ci_df

In [ ]:

cm_df_dict = {}
for method_name, confusion_matrices in cm_per_threshold.items():
    print(method_name)
    cm_df_dict_method = {}
    for threshold, cm in confusion_matrices.items():
        pr_ci_df_method = pr_ci_df[pr_ci_df['method_name'] == method_name]
        pr_ci_thresholds = pr_ci_df_method[pr_ci_df_method['threshold'] > threshold]
        try:
            index = pr_ci_thresholds['threshold'].idxmin()
            cm_df_dict_method[index]= cm
        except ValueError:
            pass
    
    cm_df_dict[method_name] = cm_df_dict_method
        
#     pr_ci_df['confusion_matrix'].loc[confusion_matrices.index] = confusion_matrices

# display(pr_ci_df)

In [ ]:
cm_df_dict

In [ ]:
# pr_ci_df[pr_ci_df['method_name'] == 'gb']['threshold'].hist()

In [ ]:
from sklearn.metrics import confusion_matrix

metrics_confusion_cohort_weighted = BinaryMetricFromScore(confusion_matrix, time='all', sample_weight=inverse_weight_cohorts)
pr = get_pr_analysis(GROUPS, y=y, metrics=[metrics_confusion_cohort_weighted], standardize=False, iterations=1)

In [ ]:
pr

## ROC analysis

In [ ]:
roc_metrics_unweighted = [
    BinaryMetricFromScore(roc_curve, time=TIME_POINT_PREDICTION, sample_weight=None),
    BinaryMetricFromScore(roc_auc_score, time=TIME_POINT_PREDICTION, sample_weight=None),
]

In [ ]:
run_roc_analysis(GROUPS, roc_metrics_unweighted, standardize=True)

In [ ]:
run_roc_analysis(GROUPS, roc_metrics_unweighted, standardize=False)

In [ ]:
inverse_weight = get_inverse_weight(data['STUDY'])

roc_metrics_weighted = [
    BinaryMetricFromScore(roc_curve, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight),
    BinaryMetricFromScore(roc_auc_score, time=TIME_POINT_PREDICTION, sample_weight=inverse_weight),
]

In [ ]:
run_roc_analysis(GROUPS, roc_metrics_weighted, standardize=True)

In [ ]:
run_roc_analysis(GROUPS, roc_metrics_weighted, standardize=False)
